In [ ]:

import os
import time
#algorihm main class to maage them
class compressionAlgorithm(object):
    def __init__(self):
        pass
    def compress(self, data):
        raise NotImplementedError
    def decompress(self, data):
        raise NotImplementedError
# Run-length encoding algorithm
class rle(compressionAlgorithm):
    def name(self):
        return 'Run-Length Encoding (RLE) no data loss algorithm, good for files with many repeating bytes'
    def compress(self, data):
        if not data:
            return b''
        compress_data = bytearray()
        count = 1 # count of the same bytes
        prev_byte = data[0] # previous byte to compare with
        for i in data[1:]: # go through the data and compare each byte with the previous one
            if prev_byte==i and count < 255:
                count += 1
            else:
                compress_data.extend([count, prev_byte]) #add the byte and its count to the compressed data
                prev_byte=i
                count = 1
        compress_data.extend([count, prev_byte]) #add the last byte and its count
        return bytes(compress_data)

    def decompress(self, data):
        if not data:
            return b''
        decompress_data = bytearray()
        for i in range(0, len(data), 2): #go through data: byte and its count at once
            if i + 1 < len(data):
                count, value = data[i], data[i + 1] #store byte and its count
                decompress_data.extend([value] * count) #add the byte count times to the decompressed data
        return bytes(decompress_data)
# Drop N bytes algorithm
class DropNbytes(compressionAlgorithm):
    def __init__(self, step=2):
        self.step = step # step to drop bytes
    def name(self):
        return f'DropNbytes_{self.step} - data loss algorithm'
    def compress(self, data):
        if not data:
            return b''
        c_data = bytearray(data)
        return bytes(c_data[::self.step]) # take every Nth byte
    def decompress(self, data):
        if not data:
            return b''
        decompressed_data = bytearray()
        for byte in data:
            decompressed_data.extend([byte]*self.step) # repeat each byte N times
        return bytes(decompressed_data)
class BlockAverageCompression(compressionAlgorithm):
    def __init__(self, block_size=4):
        self.block_size = block_size # size of the block to average
    def name(self):
        return f'BlockAverageCompression_{self.block_size} data loss algorithm'
    def compress(self, data):
        if not data:
            return b''
        compress_data = bytearray()
        for i in range(0, len(data), self.block_size): # go through data in block_size steps
            block = data[i:i+self.block_size] # block with block_size
            avg_block=sum(block)//len(block) # average of  block
            compress_data.append(avg_block) # add average to the compress data
        return bytes(compress_data)
    def decompress(self, data):
        if not data:
            return b''
        decompress_data = bytearray()
        for i in data:
            decompress_data.extend([i]*self.block_size) # add each byte block_size times
        return bytes(decompress_data)
class CompressionManager:
    def __init__(self, algorithm: compressionAlgorithm):
        self.algorithm = algorithm # lgorithm to use
    def compress(self, filename):
        with open(filename, 'rb') as f:
            data = f.read() # read file data
        print(f"algorithm: {self.algorithm.name()}")
        print(f"file: {filename}")
        t1 = time.time()
        compressed_data = self.algorithm.compress(data) # compress data
        t2 = time.time()
        print(f"Original size: {len(data)} bytes")
        print(f"Compressed size: {len(compressed_data)} bytes")
        print(f"Compression time: {t2 - t1:.6f} seconds")
        #saving files
        base, ext = os.path.splitext(filename)
        with open(f"{base}_{self.algorithm.name()}_compressed{ext}", 'wb') as f:
            f.write(compressed_data) # save compressed data
    def decompress(self, filename):
        with open(filename, 'rb') as f:
            data = f.read() # read file data
        print(f"algorithm: {self.algorithm.name()}")
        print(f"file: {filename}")
        t1 = time.time()
        decompressed_data = self.algorithm.decompress(data) # decompress data
        t2 = time.time()
        print(f"Compressed size: {len(data)} bytes")
        print(f"Decompressed size: {len(decompressed_data)} bytes")
        print(f"Decompression time: {t2 - t1:.6f} seconds")
        #saving files
        base, ext = os.path.splitext(filename)
        with open(f"{base}_{self.algorithm.name()}_decompressed{ext}", 'wb') as f:
            f.write(decompressed_data) # save decompressed data
    def compress_and_decompress(self, filename):
        with open(filename, 'rb') as f:
            data = f.read() # read file data
        print(f"algorithm: {self.algorithm.name()}")
        print(f"file: {filename}")
        t1 = time.time()
        compressed_data = self.algorithm.compress(data) # compress data
        t2 = time.time()
        decompressed_data = self.algorithm.decompress(compressed_data) # decompress data
        t3 = time.time()
        print(f"Original size: {len(data)} bytes")
        print(f"Compressed size: {len(compressed_data)} bytes")
        print(f"Compression time: {t2 - t1:.6f} seconds")
        print(f"Decompression time: {t3 - t2:.6f} seconds")
        if data == decompressed_data:
            print("True")
        else:
            print(f'error: {self.calc_error(data, decompressed_data)}')
        #saving files
        base, ext = os.path.splitext(filename)
        with open(f"{base}_{self.algorithm.name()}_compressed{ext}", 'wb') as f:
            f.write(compressed_data) # save compressed data
        with open(f"{base}_{self.algorithm.name()}_decompressed{ext}", 'wb') as f:
            f.write(decompressed_data) # save decompressed data
    def calc_error(self, data, decompress_data):
        min_len = min(len(data), len(decompress_data))
        if min_len == 0:
            return 0
        error = sum((a - b) ** 2 for a, b in zip(data, decompress_data)) / len(data) # mean squared error
        return error
def main():
    algorithms = [
        rle(),
        DropNbytes(step=2),
        BlockAverageCompression(block_size=4)
    ]
    #choose mode
    print("1. Compress")
    print('2. Decompress')
    print('3. compress + decompress')
    print('4. run all algorithms (compress+decompress)')
    number_mode = int(input('choose mode'))
    if number_mode < 1 or number_mode > 4:
        print("Invalid mode")
        return
    filename = input("Enter filename: ")
    if not os.path.isfile(filename):
        print("File not found")
        return
    if number_mode == 4:
        for alg in algorithms:
            manager = CompressionManager(alg)
            manager.compress_and_decompress(filename)
            print('-'*40)
    else:
        #choose algorithm
        for i, alg in enumerate(algorithms):
            print(f"{i + 1}. {alg.name()}")
        choice = int(input("Select algorithm: ")) - 1
        if choice < 0 or choice >= len(algorithms):
            print("Invalid choice")
            return
        manager = CompressionManager(algorithms[choice])
        if number_mode == 1:
            manager.compress(filename)
        elif number_mode == 2:
            manager.decompress(filename)
        elif number_mode == 3:
            manager.compress_and_decompress(filename)
if __name__ == "__main__":
    main()